In [1]:
import ModelTrainFilters as ModelTrain
import pickle
from time import sleep

histories = list()

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

run_type = 'super-deep-11'
# File path for the pickle file
file_path = "/home/ceg98/Documents/DaVinciVision/Model-Development/Results/" + run_type + ".pkl"
i = 2       

import tensorflow as tf
from keras import backend as K
gpu_util = .9

2023-06-05 23:40:59.270346: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 23:40:59.749311: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = gpu_util  
config.gpu_options.allow_growth = True  # Enable dynamic GPU memory allocation
sess = tf.compat.v1.Session(config=config)
K.set_session(sess)

2023-06-05 23:41:00.481262: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-05 23:41:00.497528: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-05 23:41:00.497670: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Testing Architectures

In [3]:
if not os.path.exists(file_path):
    # Save the empty list to a pickle file
    with open(file_path, "wb") as pickle_file:
        pickle.dump([], pickle_file)

In [4]:
# architectures = ['ResNet50', "BasicCNN", "AlexNet", "VGG", "DenseNet", "GoogLeNet"]

In [ ]:
lrs = [.0003, .00003]
ARCH = "ResNet50" # CONSTANT
activations = ['tanh', 'relu', 'sigmoid']
pools = ['avg', 'max', None]


# ResNet 50; Differing learning rates, image sizes, data augmentations, 


for lr in lrs:
    for acti in activations:
        for pool in pools:
            # Iterate with learning-rate, custom-gen(laplacian or garbor), weights_dir, dropout, # Do with best arch first, then with next 2,  
            tm = ModelTrain.TrainModel(architecture=ARCH, 
                                    batch_size=32, 
                                    image_size=224, 
                                    validation_split=.2, 
                                    learning_rate=lr, 
                                    seed_n=1, 
                                    verbose=1, 
                                    best_classes=True, 
                                    normalization=True,
                                    dropout=True,
                                    dropout_rate=0.1,
                                    garbor=False,
                                    laplacian=False,
                                    custom_gen=False,
                                    augments=False,
                                    short_epochs=10,
                                    full_epoch=50,
                                    full_cutoff=.45,
                                    weights_dir=run_type,
                                    model_name = ARCH+'-'+str(i),
                                    acti = acti,
                                    pool = pool) # Initializing Class




            history = tm.train()

            with open(file_path, "rb") as pickle_file:
                histories = pickle.load(pickle_file)

            histories.append(history)

            # Save the list as a pickle file
            with open(file_path, "wb") as pickle_file:
                pickle.dump(histories, pickle_file)
            
            i += 1
            if (i % 1) == 0:
                sleep(3*60)

In [10]:
import pickle
with open("/home/ceg98/Documents/DaVinciVision/Model-Development/Results/super-deep-11.pkl", "rb") as pickle_file:
    histories = pickle.load(pickle_file)

In [11]:
import pandas as pd
df = pd.DataFrame(histories)

In [12]:
df.columns

Index(['loss', 'accuracy', 'val_loss', 'val_accuracy', 'last-accuracy',
       'last-loss', 'last-val_loss', 'last-val_accuracy', 'arch', 'batch-size',
       'image-size', 'learning-rate', 'seed', 'valid_split', 'dropout',
       'dropout_rate', 'augments', 'short_epoch', 'full_epoch', 'custom_gen',
       'model_name', 'garbor', 'laplacian', 'normalization', 'best_classes',
       'features', 'activation', 'pooling'],
      dtype='object')

In [13]:
df.sort_values('last-val_accuracy', ascending=False)[['learning-rate','activation', 'pooling', 'last-val_accuracy', 'last-accuracy', 'model_name']]

,learning-rate,activation,pooling,last-val_accuracy,last-accuracy,model_name
1,0.0003,tanh,max,0.882812,0.997617,ResNet50-3
0,0.0003,tanh,avg,0.875000,0.998014,ResNet50-2
3,0.0003,relu,avg,0.875000,0.998411,ResNet50-5
2,0.0003,tanh,None,0.851562,0.997617,ResNet50-4
4,0.0003,relu,max,0.851562,0.998809,ResNet50-6
7,0.0003,sigmoid,max,0.851562,0.998014,ResNet50-9
5,0.0003,relu,None,0.820312,0.999206,ResNet50-7
6,0.0003,sigmoid,avg,0.812500,0.999206,ResNet50-8


In [14]:
df.to_csv('Results/deep-11.csv')

In [15]:
# Okay let's save the best ResNet Model. Then Also let's save the best BasicCNN & DenseNet Model. Then let's also give a run of the mill model.

In [21]:
df.columns

Index(['loss', 'accuracy', 'val_loss', 'val_accuracy', 'last-accuracy',
       'last-loss', 'last-val_loss', 'last-val_accuracy', 'arch', 'batch-size',
       'image-size', 'learning-rate', 'seed', 'valid_split', 'dropout',
       'dropout_rate', 'augments', 'short_epoch', 'full_epoch', 'custom_gen',
       'model_name', 'garbor', 'laplacian', 'normalization', 'best_classes',
       'features'],
      dtype='object')

In [28]:
import pickle
with open("/home/ceg98/Documents/DaVinciVision/Model-Development/Results/super-deep-11.pkl", "rb") as pickle_file:
    histories = pickle.load(pickle_file)
import pandas as pd
df = pd.DataFrame(histories)
df.sort_values('last-val_accuracy', ascending=False)[['last-val_accuracy', 'last-accuracy', 'model_name', 'laplacian', 'image-size']].iloc[[0]]

,last-val_accuracy,last-accuracy,model_name,laplacian,image-size
1,0.882812,0.997617,ResNet50-3,False,224


In [29]:
import pickle
with open("/home/ceg98/Documents/DaVinciVision/Model-Development/Results/base-11.pkl", "rb") as pickle_file:
    histories = pickle.load(pickle_file)
import pandas as pd
df = pd.DataFrame(histories)
df.sort_values('last-val_accuracy', ascending=False)[['last-val_accuracy', 'last-accuracy', 'image-size']].iloc[0]

last-val_accuracy      0.664062
last-accuracy          0.737887
image-size           225.000000
Name: 2, dtype: float64

In [30]:
import pickle
with open("/home/ceg98/Documents/DaVinciVision/Model-Development/Results/deep-11.pkl", "rb") as pickle_file:
    histories = pickle.load(pickle_file)
import pandas as pd
df = pd.DataFrame(histories)
df.sort_values('last-val_accuracy', ascending=False)[['last-val_accuracy', 'last-accuracy', 'model_name', 'laplacian', 'image-size']].iloc[[4, 6]]

,last-val_accuracy,last-accuracy,model_name,laplacian,image-size
25,0.707031,1.0,DenseNet-2,True,225
18,0.701823,1.0,BasicCNN3,True,225


Models to Export
1. ResNet50-3 (Super)
2. DenseNet-2
3. BasicCNN-3